In [1]:
import os
import sys

src_path = os.path.abspath('..')
print(src_path)
sys.path.append(src_path)

/home/zw12/MedLink/src


In [2]:
from utils import data_path, set_seed, create_directory

In [3]:
import csv
import json
from datetime import datetime
from collections import Counter, OrderedDict
import random
import pandas as pd
import numpy as np
from itertools import chain

In [4]:
set_seed(seed=42)

## read data

In [5]:
data_pretrain = pd.read_csv(os.path.join(data_path, 'mimic3/processed/data_pretrain.csv'))
data_pretrain

,SUBJECT_ID,GENDER,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,ICD9_FULL_CODE,ICD9_3DIGIT_CODE,CCS_CODE,AGE
0,100,F,153952,2157-08-10,2157-08-18,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,WIDOWED,UNKNOWN/NOT SPECIFIED,4260 4411 4241 42731 99602,427 426 424 996 441,106 96 237 115 105,71.0
1,1000,M,143040,2144-01-19,2144-02-25,URGENT,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,NaN,UNOBTAINABLE,MARRIED,UNKNOWN/NOT SPECIFIED,5789 2773 4280 2851 0380 40391 78559 20300 5185,203 785 277 578 038 518 285 428 403,58 60 131 99 108 153 2 40 249,69.0
2,10000,M,187813,2186-08-10,2186-08-31,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,SHORT TERM HOSPITAL,Self Pay,SPAN,CATHOLIC,NaN,HISPANIC OR LATINO,5845 28521 2766 25000 5997 5723 5712 0414 5770...,456 577 285 289 286 599 250 403 571 303 584 99...,163 157 55 660 238 152 99 64 49 134 3 211 151 ...,49.0
3,10003,M,144039,2111-11-21,2111-12-02,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,OTHER,MARRIED,BLACK/AFRICAN AMERICAN,82525 4582 2899 99811 4019 82523 8602 E8798 80708,860 458 998 825 289 E879 401 807,238 234 98 64 2616 230 231,41.0
4,10006,F,142345,2164-10-23,2164-11-01,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,3051 7850 4280 V090 25000 2874 2749 40391 5672...,305 562 285 428 996 250 403 427 E934 424 995 2...,106 99 96 108 49 2616 148 54 237 2 62 257 663 ...,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44980,9984,F,127830,2136-11-24,2136-11-27,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,7895 2720 4019 5070 00845 5849 2762 2765,272 008 584 401 789 276 507,157 53 55 98 151 129 135,80.0
44981,99897,M,162913,2181-08-06,2181-08-07,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,HAIT,7TH DAY ADVENTIST,MARRIED,BLACK/HAITIAN,28521 V1581 3572 4280 42832 25040 36201 3659 4...,780 584 V15 285 362 428 276 V58 357 365 250 40...,259 157 55 99 108 95 50 158 87 88 257 59,53.0
44982,99897,M,181057,2182-07-03,2182-07-08,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,HAIT,7TH DAY ADVENTIST,MARRIED,BLACK/HAITIAN,V5867 25052 25012 5855 3572 36201 2720 V1581 4...,272 584 V15 362 V12 795 428 V58 365 357 250 40...,259 157 53 99 108 95 50 158 87 117 10 88 257,54.0
44983,99923,M,164914,2201-02-23,2201-03-03,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,CATHOLIC,MARRIED,WHITE,5723 5859 45621 45829 5712 7455 V1582 4561 E93...,456 571 458 584 E939 303 V15 745 453 572 276 585,157 213 55 238 118 660 158 151 138 663 2617,54.0


In [6]:
for code in ['ICD9_FULL_CODE', 'ICD9_3DIGIT_CODE', 'CCS_CODE']:
    print(code)
    seqs = data_pretrain[code].tolist()
    print("max length:", max([len(seq.split(' ')) for seq in seqs]))
    create_directory(os.path.join(data_path, f'mimic3/processed/{code}'))
    with open(os.path.join(data_path, f'mimic3/processed/{code}/corpus.txt'), 'w') as f:
        for seq in seqs:
            f.write("%s\n" % seq)

ICD9_FULL_CODE
max length: 39
ICD9_3DIGIT_CODE
max length: 39
CCS_CODE
max length: 34


## run GloVe

```
cd GloVe && make
sh run.sh ../data/mimic3/processed/ICD9_FULL_CODE
sh run.sh ../data/mimic3/processed/ICD9_3DIGIT_CODE
sh run.sh ../data/mimic3/processed/CCS_CODE
```

## convert to word2vec format

In [7]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

In [8]:
for code in ['ICD9_FULL_CODE', 'ICD9_3DIGIT_CODE', 'CCS_CODE']:
    print(code)
    glove_file = datapath(os.path.join(data_path, f'mimic3/processed/{code}/vectors.txt'))
    tmp_file = get_tmpfile(os.path.join(data_path, f'mimic3/processed/{code}/word2vec.txt'))
    glove2word2vec(glove_file, tmp_file)

ICD9_FULL_CODE


/tmp/ipykernel_1694655/3096377087.py:5: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, tmp_file)


ICD9_3DIGIT_CODE
CCS_CODE
